In [46]:
#testing consequences of interventions on gpt2, and how they match up against our SAE's
import torch

feature_encoder_weights = torch.randn(768)
feature_encoder_bias = torch.randn(1)

feature_decoder_weights = torch.randn(768)
feature_decoder_bias = torch.randn(1)


In [47]:
import torch
a = torch.randn( [2,4,5])
b = torch.randn( [2, 5,1])
(a @ b).size()

torch.Size([2, 4, 1])

In [48]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Define the hook function
def strengthen_sae_feature(module, input, output):
    #identify first position where the SAE fires
    # Compute the feature activation

    #output: batch, seq, embed
    #feature_encoder_weights: 1 , embed , 1  
    # multiply result: batch, seq,
    # #we want multiply result of batch, seq, embed
    
    #feature_activation = (output @ feature_encoder_weights.unsqueeze(0).unsqueeze(2)).squeeze(1) - feature_encoder_bias
    #This way of doing it with einsum might be better
    feature_activation = torch.einsum('bse,e->bs', output[0], feature_encoder_weights) - feature_encoder_bias

    # Find the first position where the feature activates (if any)
    first_activation_positions = (feature_activation > 0).float().argmax(dim=1)

    # Check if the feature activates at all (argmax will return 0 if no activation)
    has_activation = (feature_activation > 0).any(dim=1)

    # Set positions to -1 where there's no activation
    first_activation_positions[~has_activation] = -1

    # Store the result
    global first_activation_positions_global
    first_activation_positions_global = first_activation_positions

    global store_output
    store_output = output

    # Now, in only those positions, add lambda times feature_decoder_weights to the output
    lambda_value = 1.0  # You can adjust this value as needed
    batch_size, seq_len, embed_dim = output[0].shape
    
    # Create a mask for the positions where we want to add the feature
    mask = torch.arange(seq_len).unsqueeze(0).expand(batch_size, -1) == first_activation_positions_global.unsqueeze(1)
    mask = mask.unsqueeze(-1).expand(-1, -1, embed_dim)
    
    # Add lambda times feature_decoder_weights to the output at the masked positions
    new_output = output[0] + lambda_value * mask * feature_decoder_weights.unsqueeze(0).unsqueeze(0)

    new_outputs = [new_output] + list(output[1:])
    return tuple(new_outputs)

def return_consequent_layer(module, input, output):

    # Get the batch size and sequence length from the output
    batch_size, seq_len, embed_dim = output[0].shape
    
    # Create a mask for the positions where the feature first activates
    mask = torch.arange(seq_len).unsqueeze(0).expand(batch_size, -1) == first_activation_positions_global.unsqueeze(1)
    
    # Expand the mask to match the embedding dimension
    mask = mask.unsqueeze(-1).expand(-1, -1, embed_dim)
    
    # Use the mask to zero out all positions except where the feature first activates
    filtered_output = output[0] * mask
    
    # Sum along the sequence dimension to get one embedding per batch item
    # This will effectively select the embedding at the first activation position
    # for each item in the batch
    selected_embeddings = filtered_output.sum(dim=1)
    
    # Store the selected embeddings
    global consequent_embeddings
    consequent_embeddings = selected_embeddings

    # # Store the output of the middle layer
    # global modified_layer
    # modified_layer = output


intervention_index = 5
readout_index = 8

# Register the hook on the chosen middle layer
intervention_hook = model.transformer.h[intervention_index].register_forward_hook(strengthen_sae_feature)
readout_hook = model.transformer.h[readout_index].register_forward_hook(return_consequent_layer)

#then we have to read out the modified layer! will be stored under consequent_embeddings, size: batch , embed

# Example usage
text = ["Hello, world!", "Hello, world!"]
inputs = tokenizer(text, return_tensors="pt")

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)

# Now middle_layer_output contains the output of the middle layer
print(f"shape of output embeddings: {consequent_embeddings.shape}")


# Remove the hook after use
intervention_hook.remove()
readout_hook.remove()


shape of output embeddings: torch.Size([2, 768])


In [49]:
first_activation_positions_global

tensor([1, 1])

In [50]:
consequent_embeddings

tensor([[ 0.8405, -0.1911,  1.7396,  ..., -2.1171,  0.0240,  1.9667],
        [ 0.8405, -0.1911,  1.7396,  ..., -2.1171,  0.0240,  1.9667]])

In [ ]:
#next code we would need to add:
#filter out any batch elements where the SAE doesn't trigger
#compare to activations from a hook on the clean sequence
#subtract, compute comparisons, etc.!